# Fruit Rottenness Detection Using CNN

## Import Statements

In [ ]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import Sequence


## Setting the path to dataset

In [ ]:
# Set the path to your dataset
train_data_dir = 'dataset/training_set'
test_data_dir = 'dataset/test_set'
img_width, img_height = 40, 40
batch_size = 60
epochs = 15


## Defining functions for Preprocessing Image

### Segmentation (K-Means Clustering)

In [ ]:
def custom_preprocessing(image, k=4, max_iterations=100, epsilon=0.2):
    # Reshape the image to a 2D array of pixels
    pixels = image.reshape((-1, 3))

    # Convert pixels to float32
    pixels = pixels.astype(np.float32)

    # Initialize k centroids randomly
    centroids = pixels[np.random.choice(pixels.shape[0], k, replace=False)]

    for _ in range(max_iterations):
        # Calculate distances between each pixel and centroids
        distances = np.linalg.norm(pixels[:, np.newaxis, :] - centroids, axis=2)

        # Assign each pixel to the closest centroid
        labels = np.argmin(distances, axis=1)

        # Update centroids based on the mean of pixels in each cluster
        for i in range(k):
            cluster_pixels = pixels[labels == i]
            if len(cluster_pixels) > 0:
                centroids[i] = np.mean(cluster_pixels, axis=0)

    # Assign each pixel to the final centroids
    segmented_image = centroids[labels]
    
    # Reshape the segmented image to the original shape
    segmented_image = segmented_image.reshape(image.shape)

    return segmented_image.astype('float32')




### Grayscale

In [ ]:
# Creating Grayscale Version of the Image
def custom_preprocessing2(image):
    # Extract the RGB channels
    red_channel = image[:, :, 0]
    green_channel = image[:, :, 1]
    blue_channel = image[:, :, 2]

    # Convert to grayscale using the formula: grayscale = 0.299 * R + 0.587 * G + 0.114 * B
    grayscale_image = 0.299 * red_channel + 0.587 * green_channel + 0.114 * blue_channel

    # Expand dimensions to make it compatible with the model's input shape
    grayscale_image = np.expand_dims(grayscale_image, axis=-1)

    return grayscale_image.astype('float32')

### Edge Detection

In [ ]:

def custom_preprocessing3(image):
    # Convert the image to grayscale
    grayscale_image = rgb2gray(image)

    # Define Sobel kernels for horizontal and vertical edge detection
    kernel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]])

    kernel_y = np.array([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]])

    # Apply convolution with the Sobel kernels
    sobel_x = convolve2d(grayscale_image, kernel_x, mode='same', boundary='symm')
    sobel_y = convolve2d(grayscale_image, kernel_y, mode='same', boundary='symm')

    # Calculate the gradient magnitude
    gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)

    # Normalize the gradient magnitude to the range [0, 255]
    normalized_gradient = (gradient_magnitude / gradient_magnitude.max()) * 255

    # Convert to uint8 type
    edges = normalized_gradient.astype(np.uint8)

    # Expand dimensions to make it compatible with the model's input shape
    edges = np.expand_dims(edges, axis=-1)

    return edges.astype('float32')


### Noise Addition

In [ ]:
def custom_preprocessing4(image, noise_intensity=75):
    # Convert the image to a NumPy array
    image_array = np.array(image)

    # Get the shape of the image
    height, width, channels = image_array.shape

    # Generate random noise manually
    noise = np.random.randint(-noise_intensity, noise_intensity, (height, width, channels))

    # Add noise to the original image and clip to [0, 255]
    noisy_image = np.clip(image_array + noise, 0, 255).astype('uint8')

    return noisy_image.astype('float32')

### Shading

In [ ]:
#Applying Shading
def custom_preprocessing5(image):
    gamma=3
    shaded_image = np.clip(np.power(image / 255.0, gamma) * 255.0, 0, 255).astype('uint8')
    
    return shaded_image.astype('float32')


## Data Preprocessing

### Creating different generators representing each of the preprocessing function

In [ ]:

# Data Preprocessing
train_datagen1 = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
#     vertical_flip=True,
    preprocessing_function=custom_preprocessing
)

# DirectoryIterator for the dataset
train_generator1 = train_datagen1.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:

train_datagen2 = ImageDataGenerator(
     rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
#     vertical_flip=True,
    preprocessing_function=custom_preprocessing2
)

train_generator2 = train_datagen2.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:

train_datagen3 = ImageDataGenerator(
     rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
#     vertical_flip=True,    
    preprocessing_function=custom_preprocessing3
)

train_generator3 = train_datagen3.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


In [ ]:
train_datagen4 = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
#     vertical_flip=True,    
    preprocessing_function=custom_preprocessing4
)

train_generator4 = train_datagen4.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


In [ ]:
train_datagen5 = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
#     vertical_flip=True,    
    preprocessing_function=custom_preprocessing5
)

train_generator5 = train_datagen5.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


### Creating the test generator

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1.0/255)

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

## Concatenation of all the Generators

### Creating the class for combining all the generators

In [ ]:

class ConcatenateGenerators(Sequence):
    def __init__(self, generators):
        self.generators = generators
        self.lengths = [len(gen) for gen in generators]
        self.cumulative_lengths = np.cumsum(self.lengths)

    def __len__(self):
        return self.cumulative_lengths[-1]

    def __getitem__(self, index):
    # Initialize empty lists to store data and labels from all generators
        all_data, all_labels = [], []

        for generator_index, generator in enumerate(self.generators):
            if index < self.cumulative_lengths[generator_index]:
                # Calculate the sample index within the current generator
                sample_index = index if generator_index == 0 else index - self.cumulative_lengths[generator_index - 1]

                # Get the data and labels from the current generator
                data, labels = generator[sample_index]

                # Append data and labels to the lists
                all_data.append(data)
                all_labels.append(labels)

        # Concatenate data and labels from all generators
        data = np.concatenate(all_data, axis=0)
        labels = np.concatenate(all_labels, axis=0)

        return data, labels


### Combination of multiple generators into a single generator

In [ ]:

combined_generator = ConcatenateGenerators([train_generator1, train_generator2, train_generator3, train_generator4, train_generator5])

## Building the CNN

### Defining the layers

In [ ]:

# Initialising the CNN
model = tf.keras.models.Sequential()

# Step 1 - Convolution
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[40, 40, 3]))
# model.add(tf.keras.layers.BatchNormalization())

# Step 2 - Pooling
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# model.add(tf.keras.layers.BatchNormalization())

# Adding a second convolutional layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# model.add(tf.keras.layers.BatchNormalization())

# Step 3 - Flattening
model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.BatchNormalization())

# Step 4 - Full Connection
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
# model.add(tf.keras.layers.BatchNormalization())


# Step 5 - Output Layer
model.add(tf.keras.layers.Dense(units=8, activation='softmax'))


### Compiling the model

In [ ]:
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#                 initial_learning_rate = 1e-3,
#                 decay_steps = 10000,
#                 decay_rate = 0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


### Training the CNN model

In [ ]:

history = model.fit(combined_generator, epochs=epochs, validation_data=test_generator)


In [ ]:
from tabulate import tabulate

train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

table_data = [(epoch + 1, train_acc,f"{train_acc * 100:.2f}%", val_acc, f"{val_acc * 100:.2f}%") for epoch, (train_acc, val_acc) in enumerate(zip(train_accuracy, val_accuracy))]

headers = ['Epoch', 'Train Accuracy', 'Train Accuracy (%)', 'Validation Accuracy', 'Validation Accuracy (%)']

print(tabulate(table_data, headers=headers, tablefmt='grid'))


## Plotting the graph

In [ ]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Show the plots
plt.tight_layout()
plt.show()

model.save('fruit_classifier_model101.h5')


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import cv2
from sklearn.metrics import f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Load the pre-trained model
model_path = 'fruit_classifier_model.h5'
model = load_model(model_path)

# Set the path to your test dataset
test_data_dir = 'dataset/test_set'
img_width, img_height = 35, 35
batch_size = 32

# Data Preprocessing for testing set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=False)  # Ensure labels are in the same order as predictions

# Get true labels
true_labels = test_generator.classes

# Make predictions
predictions = model.predict(test_generator)

# Get predicted labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate Confusion Matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Display F1 score
print(f"F1 Score: {f1:.4f}")

# Display Confusion Matrix with rotated x-axis labels
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices, yticklabels=test_generator.class_indices, cbar=True)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels
plt.show()


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Load the pretrained model
model_path = 'fruit_classifier_model1.h5'
model = load_model(model_path)

# Function to predict the class of an image
def predict_image(image_path):
    img = load_img(image_path, target_size=(35, 35))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    classes = {0: 'fresh_apple', 1: 'fresh_banana', 2: 'fresh_orange', 3: 'fresh_strawberry', 4: 'rotten_apple', 5: 'rotten_banana', 6: 'rotten_orange', 7: 'rotten_strawberry'}

    # Normalize the probabilities to ensure the sum is 100%
    normalized_probabilities = predictions[0] / np.sum(predictions[0])

    # Print class probabilities
    for i in range(len(normalized_probabilities)):
        print(f"{classes[i]}: {normalized_probabilities[i] * 100:.7f} %")

    # Get the predicted class using the normalized probabilities
    predicted_class_index = np.argmax(normalized_probabilities)
    predicted_class = classes[predicted_class_index]

    print(f"Predicted class: {predicted_class}")

if __name__ == "__main__":
    # Example usage
    image_path = 'dataset2/single_prediction/fruit_test_1.jpg'
    predict_image(image_path)


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Load the pretrained model
model_path = 'fruit_classifier_model1.h5'
model = load_model(model_path)

# Function to predict the class of an image
def predict_image(image_path):
    img = load_img(image_path, target_size=(35, 35))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    classes = {0: 'fresh_apple', 1: 'fresh_banana', 2: 'fresh_orange', 3: 'fresh_strawberry', 4: 'rotten_apple', 5: 'rotten_banana', 6: 'rotten_orange', 7: 'rotten_strawberry'}

    # Normalize the probabilities to ensure the sum is 100%
    normalized_probabilities = predictions[0] / np.sum(predictions[0])

    # Plotting the bar diagram with rotated x-axis labels
    plt.figure(figsize=(10, 6))
    bars = plt.bar(classes.values(), normalized_probabilities, color=['green' if p >= 0.5 else 'red' for p in normalized_probabilities])
    plt.title('Class Probabilities')
    plt.xlabel('Classes')
    plt.ylabel('Probability')
    plt.ylim(0, 1)  # Set the y-axis limit to be between 0 and 1 for probabilities
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
    plt.tight_layout()  # Adjust layout for better spacing
    plt.show()

    # Get the predicted class using the normalized probabilities
    predicted_class_index = np.argmax(normalized_probabilities)
    predicted_class = classes[predicted_class_index]

    print(f"\nPredicted class: {predicted_class}")

if __name__ == "__main__":
    # Example usage
    image_path = 'dataset2/single_prediction/rotten_apple_test1.jpg'
    predict_image(image_path)


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
model_path = 'fruit_classifier_model1.h5'
model = load_model(model_path)


In [ ]:
model.summary()